<a href="https://colab.research.google.com/github/jaoaguiar097/EngenhariaIA/blob/main/calmariasspas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wellness Data Pipeline
Pipeline simples para tratamento e qualidade de dados de bem-estar.


**Um pipeline simples que:**

Lê documentos de texto (simulando PDFs e feedbacks)

Limpa ruído (lixo de texto)

Divide em pedaços (chunks)

Enriquece com metadados

Prepara os dados para uso em IA (ex: busca semântica)



**Stack (simples e realista)**

Python

Google Colab

pandas

nltk

scikit-learn




**Estrutura mental do pipeline**

Pensa assim:

Entrada de dados

Limpeza

Padronização

Chunking

Enriquecimento

Saída organizada

Pipeline não é moda. É disciplina.


**Importar bibliotecas**

In [1]:
import pandas as pd
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


pandas: organizar dados

re: limpar texto

nltk: remover palavras inúteis

TF-IDF: preparar texto para IA

**Criar dados de exemplo (simulando PDFs e feedbacks)**


In [2]:
data = {
    "fonte": ["manual_pdf", "feedback_cliente", "manual_pdf"],
    "texto": [
        "Tratamento relaxante com óleos essenciais. Duração: 60 minutos.",
        "Achei o atendimento ótimo!!! Muito relaxante :)",
        "Massagem terapêutica indicada para estresse e dores musculares."
    ]
}

df = pd.DataFrame(data)
df


,fonte,texto
0,manual_pdf,Tratamento relaxante com óleos essenciais. Dur...
1,feedback_cliente,Achei o atendimento ótimo!!! Muito relaxante :)
2,manual_pdf,Massagem terapêutica indicada para estresse e ...


**Função de limpeza de texto**

In [11]:
stop_words = set(stopwords.words('portuguese'))

def limpar_texto(texto):
    texto = texto.lower()
    texto = re.sub(r'[^a-zà-ú\s]', '', texto)
    palavras = texto.split()
    palavras = [p for p in palavras if p not in stop_words]
    return " ".join(palavras)

df["texto_limpo"] = df["texto"].apply(limpar_texto)
df


,fonte,texto,texto_limpo,chunks
0,manual_pdf,Tratamento relaxante com óleos essenciais. Dur...,tratamento relaxante óleos essenciais duração ...,[tratamento relaxante óleos essenciais duração...
1,feedback_cliente,Achei o atendimento ótimo!!! Muito relaxante :),achei atendimento ótimo relaxante,[achei atendimento ótimo relaxante]
2,manual_pdf,Massagem terapêutica indicada para estresse e ...,massagem terapêutica indicada estresse dores m...,"[massagem terapêutica indicada estresse dores,..."


**Chunking (dividir texto)**

In [12]:
def chunk_text(text, size=5):
    words = text.split()
    return [" ".join(words[i:i+size]) for i in range(0, len(words), size)]

df["chunks"] = df["texto_limpo"].apply(chunk_text)
df


,fonte,texto,texto_limpo,chunks
0,manual_pdf,Tratamento relaxante com óleos essenciais. Dur...,tratamento relaxante óleos essenciais duração ...,[tratamento relaxante óleos essenciais duração...
1,feedback_cliente,Achei o atendimento ótimo!!! Muito relaxante :),achei atendimento ótimo relaxante,[achei atendimento ótimo relaxante]
2,manual_pdf,Massagem terapêutica indicada para estresse e ...,massagem terapêutica indicada estresse dores m...,"[massagem terapêutica indicada estresse dores,..."


**Explodir os chunks (boa prática)**

In [7]:
df_chunks = df.explode("chunks").reset_index(drop=True)
df_chunks


,fonte,texto,texto_limpo,chunks
0,manual_pdf,Tratamento relaxante com óleos essenciais. Dur...,tratamento relaxante óleos essenciais duração ...,tratamento relaxante óleos essenciais duração
1,manual_pdf,Tratamento relaxante com óleos essenciais. Dur...,tratamento relaxante óleos essenciais duração ...,minutos
2,feedback_cliente,Achei o atendimento ótimo!!! Muito relaxante :),achei atendimento ótimo relaxante,achei atendimento ótimo relaxante
3,manual_pdf,Massagem terapêutica indicada para estresse e ...,massagem terapêutica indicada estresse dores m...,massagem terapêutica indicada estresse dores
4,manual_pdf,Massagem terapêutica indicada para estresse e ...,massagem terapêutica indicada estresse dores m...,musculares


**Enriquecer com metadados**

In [14]:
df_chunks["tipo_conteudo"] = df_chunks["fonte"].apply(
    lambda x: "manual" if "manual" in x else "feedback")
df_chunks


,fonte,texto,texto_limpo,chunks,tipo_conteudo
0,manual_pdf,Tratamento relaxante com óleos essenciais. Dur...,tratamento relaxante óleos essenciais duração ...,tratamento relaxante óleos essenciais duração,manual
1,manual_pdf,Tratamento relaxante com óleos essenciais. Dur...,tratamento relaxante óleos essenciais duração ...,minutos,manual
2,feedback_cliente,Achei o atendimento ótimo!!! Muito relaxante :),achei atendimento ótimo relaxante,achei atendimento ótimo relaxante,feedback
3,manual_pdf,Massagem terapêutica indicada para estresse e ...,massagem terapêutica indicada estresse dores m...,massagem terapêutica indicada estresse dores,manual
4,manual_pdf,Massagem terapêutica indicada para estresse e ...,massagem terapêutica indicada estresse dores m...,musculares,manual


**Vetorização (pré-IA)**

In [10]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df_chunks["chunks"])

X.shape


(5, 15)